# TODO

- Convert classes from text to tensor array with size 100
- Complete train loop (it is not working now) due to problem above
- Find good LSTM architecture
- Check how fast is our approach with per frame check

## Web cum test

In [4]:
import time
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.PoseModule import PoseDetector


def webcum():
    cap = cv2.VideoCapture(0)
    hands_detector = HandDetector()
    pose_detector = PoseDetector()
    while True:
        success, img = cap.read()
        hands, img1 = hands_detector.findHands(img)
        print(hands)
        img2 = pose_detector.findPose(img)
        lmList, bboxInfo = pose_detector.findPosition(img, bboxWithHands=False)
        print(lmList)
        print()
        time.sleep(2)
        # cv2.imshow("CUM", img)
        # cv2.waitKey(1)

## Device Stuff

In [28]:
import torch

#Set up device
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cpu


## Dataset

In [3]:
import json
import os
import torch
from torch.utils.data import Dataset, DataLoader
from cvzone.HandTrackingModule import HandDetector
from cvzone.PoseModule import PoseDetector
import cv2

In [34]:
class ASLDataset(Dataset):
    """
    A custom dataset class for loading American Sign Language (ASL) videos and their corresponding labels.

    Args:
        arrays_folder (str): Path to the folder containing the .pt arrays (converted videos).
        name_with_label (dict): A dictionary mapping video names to their labels.
        classes (list): A list of class names.

    Attributes:
        arrays_folder (str): Path to the folder containing the .pt arrays (converted videos).
        name_with_label (dict): A dictionary mapping video names to their labels.
        arrays_names (list): A list of .pt tensor arrays names.
        classes (list): A list of class names.
    """

    def __init__(self, arrays_folder, name_with_label, classes, max_frame_count):
        self.arrays_folder = arrays_folder
        self.name_with_label = name_with_label
        self.arrays_names = list(self.name_with_label.keys())
        self.classes = classes
        self.max_frame_count = max_frame_count

        self.default_frame = [0] * (21 * 3 * 2 + 33 * 3)
        # Load videos
        self.tensor_sequences = self._load_all_videos()

    def _load_all_videos(self):
        """
        Loads all tensors in advance.

        Returns:
            list[torch.tensor]: List of tensors.
        """

        tensor_sequences = []
        
        for index in range(len(self.arrays_names)):
            array_path = self.arrays_folder + '/' + self.arrays_names[index] + '.pt'
            # Open the video file using OpenCV
            tensor = torch.load(array_path)
            
            pad_right = [self.default_frame for _ in range(0, self.max_frame_count - tensor.shape[0])]
            pad_right = torch.tensor(pad_right)
            tensor_sequences.append(torch.cat((tensor, pad_right)))

        return tensor_sequences
        
    def __len__(self):
        """
        Returns the number of videos in the dataset.

        Returns:
            int: The number of videos in the dataset.
        """
        return len(self.arrays_names)

    def __getitem__(self, index):
        """
        Retrieves a video and its corresponding label from the dataset.

        Args:
            index (int): The index of the video to retrieve.

        Returns:
            tuple: A tuple containing the video frames as a PyTorch tensor and the label.
        """


        return self.tensor_sequences[index].float(), self.name_with_label[self.arrays_names[index]][0]


In [35]:
def read_preprocess_json(json_name, videos_root):
    """
    Read json and separate videos on predefined subsets(train, val, test). Check for existence of videos.

    :param json_name: path or name of json file in format {'video_name.mp4': {'subset': 'train', 'action': [class_num,
    start_frame, end_frame]}}
    :param videos_root: root folder of all videos
    :return: train, validation and test dictionaries in format {'video_name.mp4': [class_num, start_frame, end_frame]}
    """
    videos = json.load(open(json_name))
    train, val, test = dict(), dict(), dict()
    for name in os.listdir(videos_root):
        name = name[:-4]
        if videos.get(name) is None:
            continue
        if videos[name]['subset'] == 'train':
            train[name] = videos[name]['action']
        elif videos[name]['subset'] == 'val':
            val[name] = videos[name]['action']
        elif videos[name]['subset'] == 'test':
            test[name] = videos[name]['action']
    return train, val, test

In [36]:
def read_classes(path):
    classes = dict()
    with open(path, 'r') as file:
        for line in file:
            line = line.strip().split('\t')
            key = int(line[0])
            value = line[1]
            classes[key] = value
    return classes

In [37]:
def get_maximum_frame_count(arrays_root):
    max_len = 0
    for name in os.listdir(arrays_root):
        full_path = os.path.join(arrays_root, name)
        tensor = torch.load(full_path)
        max_len = max(max_len, tensor.shape[0])
    return max_len
max_frame_count = get_maximum_frame_count("point_arrays")
print(f"Maximum length of video in frames in current dataset is {max_frame_count}")

Maximum length of video in frames in current dataset is 195


In [62]:
train, val, test = read_preprocess_json('wlasl_dataset/nslt_100.json', 'wlasl_dataset/videos')
classes = read_classes('wlasl_dataset/wlasl_class_list.txt')

train_dataset = ASLDataset('point_arrays', train, classes, max_frame_count)
val_dataset = ASLDataset('point_arrays', val, classes, max_frame_count)
test_dataset = ASLDataset('point_arrays', test, classes, max_frame_count)

2000


In [39]:
print(train_dataset[1][0].shape)

torch.Size([195, 225])


In [40]:
# Create dataloaders

batch_size = 4

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Model

In [57]:
import torch.nn as nn

class SequenceModel(nn.Module):
    def __init__(self, input_size=1662) -> None:
        super().__init__()
        self.lstm = nn.Sequential(
            nn.LSTM(input_size, 256, 3, batch_first=True), # TODO: Разобраться с LSTM в pytorch
        )
        self.linear = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 100),
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)

        return x

In [58]:
model = SequenceModel(input_size=225)
model

SequenceModel(
  (lstm): Sequential(
    (0): LSTM(225, 64, num_layers=3, batch_first=True)
  )
  (linear): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): Softmax(dim=1)
  )
)

## Training

In [59]:
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm


def train_model(model: nn.Module, epochs: int, criterion, optimizer):
    """
    Function that trains model using number of epochs, loss function, optimizer.
    Can use validation or test data set for evaluation.
    Calculates f1 score.

    Parameter
    ---------
    model : nn.Module
      Model to train.
    epochs: int
      Number of train epochs
    criterion
      The loss function from pytorch
    optimizer
      The optimizer from pytorch
    """

    # Train the model
    for epoch in range(epochs):
        model.train()

        #
        predicted_train = []
        true_train = []

        train_loss = 0.0

        bar = tqdm(train_dataloader)
        iterations = 0

        for inputs, outputs in bar:
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            predictions = model(inputs.to(device))
            loss = criterion(predictions, outputs.to(device))
            train_loss += loss.item()

            # Backward pass
            loss.backward()
            optimizer.step()

            # Get predicted classes and true classes from data
            for item in predictions:
                predicted_train.append(int(torch.argmax(item).cpu().numpy()))
            for item in outputs:
                true_train.append(int(torch.argmax(item).cpu().numpy()))
            iterations += 1
            bar.set_postfix(({"loss": f"{train_loss/(iterations*train_dataloader.batch_size)}"}))

        # Computing loss
        train_loss /= len(train_dataset)
        # Computing f1 score
        train_f1 = f1_score(true_train, predicted_train, average="macro")

        # Printing information in the end of train loop
        test_loss, test_f1 = test_model(model, criterion, test_dataloader)
        print(f"Epoch {epoch+1} train (loss: {train_loss:.4f}, f1 score: {train_f1:.4f}) test (loss: {test_loss:.4f}, f1 score: {test_f1:.4f})")


def test_model(model: nn.Module, criterion, test_dataloader: DataLoader):
    """
    Function that evaluates model on specified dataloader
    by specified loss function.

    Parameter
    ---------
    model : nn.Module
      Model to train.
    criterion
      The loss function from pytorch
    test_dataloader: DataLoader
      The dataset for testing model

    Returns
    -------
    float: loss of model on given dataset
    float: f1 score of model on given dataset
    """

    model.eval()

    # Test loss value
    test_loss = 0.0

    # Lists for calculation f1 score
    predicted_test = []
    true_test = []

    with torch.no_grad():
        for inputs, outputs in test_dataloader:

            # Forward pass
            predictions = model(inputs.to(device))
            test_loss += criterion(predictions, outputs.to(device))

            # Get predicted classes and true classes from data
            for item in predictions:
                predicted_test.append(int(torch.argmax(item).cpu().numpy()))
            for item in outputs:
                true_test.append(int(torch.argmax(item).cpu().numpy()))

    # Computation of test loss
    test_loss /= len(test_dataloader)

    # Computation of f1 score
    test_f1 = f1_score(true_test, predicted_test, average="macro")
    test_accuracy = accuracy_score(true_test, predicted_test)
    return test_loss.item(), test_f1, test_accuracy


In [60]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_model(model, 10, criterion, optimizer)

  0%|                                                                                          | 0/187 [00:00<?, ?it/s]


RuntimeError: Expected target size [4, 32], got [4]